In [39]:
from __future__ import print_function
from pandas import read_csv

import random 
import math
import numpy as np
import pandas as pd

from random import randint
import matplotlib.pyplot as plt
dataset = read_csv('/home/aiying/Machinelearning/dataorigin.csv')

headers=list(dataset)
ds=dataset.values.tolist()  
nset=[]

for i in headers:
    indexNames = dataset[i].index[dataset[i].apply(np.isnan)]
    if len(indexNames)==0:
         continue
    set=[]   
    for i in list(indexNames):
        set.append(ds[i])
    nset.append(set)   


limit=int(144/5)
validrow=[]
rowsum=list(dataset.isnull().sum(axis=1))
for i in range(len(rowsum)):
    if rowsum[i]<limit:
        validrow.append(i)    

validds=[]
for i in validrow:
    validds.append(ds[i])    


order=[]
for i in range(41):
    order=order+random.sample(range(0,144), 144)    

    n=0
for row in validds[0:len(validds)]:
    while math.isnan(row[order[n]]):
        order[n]=(order[n]+1)%144
    row[order[n]]=np.NaN  
    validds.append(row)
    n=n+1

validdataset=pd.DataFrame(validds)

headers = list(validdataset)
ds=validdataset.values.tolist()              
 
modset=[]
modframe=[]
modframelen=[]
testcolumnameset=[]
testcolumnset=[]

j=-1
for i in headers:
     j=j+1
     indexNames = validdataset[i].index[validdataset[i].apply(np.isnan)]
     if len(indexNames)==0:
         continue
     newds=validdataset.drop(indexNames)
     modframe.append(newds)
     lnewds=newds.values.tolist()
     modset.append(lnewds)
     modframelen.append(len(lnewds))
     testcolumnameset.append(i)
     testcolumnset.append(j)
print('sum of columns have missing data', len(modset))
print('shortest column',min(modframelen))


sum of columns have missing data 144
shortest column 4976


In [40]:
len(nset[0])

3403